In [1]:
# =================================================================
# TITULO: 2. SETUP DE ENTRENAMIENTO Y FEATURE ENGINEERING (ATHOS)
# UCAS: Aprendizaje de Máquinas, Minería de Datos (Grafos)
# OBJETIVO: Entrenar el modelo XGBoost con features de riesgo PageRank
# =================================================================

## 1. Importación de Módulos Esenciales

import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

# --- CORRECCIÓN DE IMPORTACIÓN ---
# Si ejecutas el Notebook desde 'notebooks/', solo necesitamos asegurar la ruta 'src'
import sys
import os
# Añadir la carpeta raíz del proyecto al path (un nivel arriba)
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir))) 

# Ahora importamos usando la estructura de carpetas: src.nombre_archivo
from src.data_pipeline import simulate_historical_data
from src.ml_features import build_transaction_graph, calculate_pagerank_feature
# ----------------------------------

print("✅ Módulos cargados exitosamente.")

## 2. Carga y Preparación de Datos (Fase Big Data)

# Cargar el DataFrame base de transacciones
df = simulate_historical_data()
print(f"Datos cargados. Tamaño inicial: {df.shape}")
print(f"Distribución inicial de Fraude (y): \n{df['is_fraud'].value_counts()}")

# Separar Features (X) y Target (y)
X = df.drop('is_fraud', axis=1)
y = df['is_fraud']
# Eliminamos IDs que no son features de ML directo
X = X.drop(['transaction_id'], axis=1)

## 3. Feature Engineering de Redes (Minería de Datos / Grafos)

print("\n--- INICIANDO ANÁLISIS DE GRAFOS (Monitoreo Continuo AML) ---")

# Tarea 1: Construir el Grafo de Transacciones
G = build_transaction_graph(df)
print(f"Grafo construido con {G.number_of_nodes()} nodos y {G.number_of_edges()} aristas.")

# Tarea 2: Calcular el PageRank (Feature de Influencia de Nodos)
# PageRank es la Feature clave de Minería de Datos para el ML.
df_pagerank = calculate_pagerank_feature(G)
df_pagerank.rename(columns={'customer_id': 'sender_id'}, inplace=True) # Renombrar para unir

# Unir la nueva feature al DataFrame principal (X)
X = pd.merge(X, df_pagerank, on='sender_id', how='left')

# Imputar valores nulos (cuentas sin PageRank, ej. nuevas cuentas)
X['pagerank_score'].fillna(X['pagerank_score'].mean(), inplace=True)

# Finalizar la preparación de X para el ML
X = pd.get_dummies(X, columns=['sender_id', 'receiver_id'])

print("Feature de PageRank calculada y añadida a X.")

## 4. Balanceo de Clases (Aprendizaje de Máquinas - SMOTE)

# Esto es CRÍTICO para reducir Falsos Negativos (fraudes no detectados)
print("\n--- APLICANDO SMOTE PARA BALANCEO DE CLASES ---")
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)

print(f"Distribución después de SMOTE: \n{y_res.value_counts()}")

## 5. Entrenamiento y Evaluación del Modelo XGBoost

# Dividir datos
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)

# Entrenar XGBoost (Modelo robusto requerido)
model = XGBClassifier(n_estimators=100, learning_rate=0.1, use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

# Evaluación de rendimiento
y_pred = model.predict(X_test)
print("\n--- REPORTE DE CLASIFICACIÓN (Métricas ML) ---")
print(classification_report(y_test, y_pred))

# Almacenar los datos de prueba y las probabilidades para el Demo Interactivo
# Guardamos los datos originales de prueba para la simulación de Falsos Positivos
X_test_orig = X[X.index.isin(y_test.index)] 
y_proba = model.predict_proba(X_test_orig)[:, 1] # Probabilidades de fraude

## 6. Guardar Modelo y Datos de Prueba (Conexión al Demo Interactivo)

# Exportar el modelo y los datos necesarios para el demo interactivo
MODEL_FILE = '../model_artifacts/xgboost_model.pkl'
DATA_FILE = '../model_artifacts/demo_data.pkl'

os.makedirs('../model_artifacts', exist_ok=True)

joblib.dump(model, MODEL_FILE)
joblib.dump({'X_test': X_test_orig, 'y_proba': y_proba}, DATA_FILE)

print(f"\n✅ Proceso de entrenamiento COMPLETADO.")
print(f"Modelo guardado en: {MODEL_FILE}")

✅ Módulos cargados exitosamente.
Datos cargados. Tamaño inicial: (5000, 5)
Distribución inicial de Fraude (y): 
is_fraud
0    4912
1      88
Name: count, dtype: int64

--- INICIANDO ANÁLISIS DE GRAFOS (Monitoreo Continuo AML) ---
Grafo construido con 899 nodos y 4989 aristas.
Feature de PageRank calculada y añadida a X.

--- APLICANDO SMOTE PARA BALANCEO DE CLASES ---


C:\Users\diego\AppData\Local\Temp\ipykernel_17476\3593040448.py:61: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X['pagerank_score'].fillna(X['pagerank_score'].mean(), inplace=True)


Distribución después de SMOTE: 
is_fraud
1    4912
0    4912
Name: count, dtype: int64


c:\Users\diego\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:199: UserWarning: [23:34:20] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



--- REPORTE DE CLASIFICACIÓN (Métricas ML) ---
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      1000
           1       1.00      0.99      0.99       965

    accuracy                           0.99      1965
   macro avg       0.99      0.99      0.99      1965
weighted avg       0.99      0.99      0.99      1965


✅ Proceso de entrenamiento COMPLETADO.
Modelo guardado en: ../model_artifacts/xgboost_model.pkl
